# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
from sklearn import linear_model, cross_validation
%matplotlib inline

In [104]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [105]:
level1prestige = sum((df.admit == 1) & (df.prestige == 1.0))
level2prestige = sum((df.admit == 1) & (df.prestige == 2.0))
level3prestige = sum((df.admit == 1) & (df.prestige == 3.0))
level4prestige = sum((df.admit == 1) & (df.prestige == 4.0))
print level4prestige, level3prestige, level2prestige, level1prestige

12 28 53 33


In [32]:
# frequency table for prestige and whether or not someone was admitted
persadmit = 0
persnoadmit = 0
prestige4 = 0
prestige3 = 0
prestige2 = 0
prestige1 = 0
for admitperson in df['admit']:
    if admitperson == 1 :
        persadmit +=1
    elif admitperson == 0:
        persnoadmit +=1
for prestigeperson in df['prestige']:
    if prestigeperson > 3:
        prestige4 +=1
    elif prestigeperson > 2:
        prestige3 +=1
    elif prestigeperson > 1:
        prestige2 +=1
    else:
        prestige1 +=1

print "admittance granted to " + str(persadmit) + " and non-admittance to " + str(persnoadmit) + " prestige4 level for " + str(prestige4) + ",prestige3 level for " + str(prestige3) + ",prestige2 level for " + str(prestige2) + ",and prestige1 level for " + str(prestige1)


# frequency table for prestige and whether or not someone was admitted
df.ix[:,('prestige',)] = df.prestige.astype(int)
df.ix[:,('admit',)] = df.admit.astype(int)
df['combined'] = df.apply(lambda row: 'P:{:.0f} A:{}'.format(row.prestige, row.admit), axis = 1)
df.combined.value_counts()

admittance granted to 126 and non-admittance to 271 prestige4 level for 67,prestige3 level for 121,prestige2 level for 148,and prestige1 level for 61


/Users/figure8labs/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


P:2 A:0    95
P:3 A:0    93
P:4 A:0    55
P:2 A:1    53
P:1 A:1    33
P:1 A:0    28
P:3 A:1    28
P:4 A:1    12
Name: combined, dtype: int64

## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [29]:
dummy_ranks = df.join(pd.get_dummies(df.prestige, prefix='P'))
dummy_ranks

,admit,gre,gpa,prestige,combined,P_1,P_2,P_3,P_4
0,0,380.0,3.61,3,P:3 A:0.0,0.0,0.0,1.0,0.0
1,1,660.0,3.67,3,P:3 A:1.0,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1,P:1 A:1.0,1.0,0.0,0.0,0.0
3,1,640.0,3.19,4,P:4 A:1.0,0.0,0.0,0.0,1.0
4,0,520.0,2.93,4,P:4 A:0.0,0.0,0.0,0.0,1.0
5,1,760.0,3.00,2,P:2 A:1.0,0.0,1.0,0.0,0.0
6,1,560.0,2.98,1,P:1 A:1.0,1.0,0.0,0.0,0.0
7,0,400.0,3.08,2,P:2 A:0.0,0.0,1.0,0.0,0.0
8,1,540.0,3.39,3,P:3 A:1.0,0.0,0.0,1.0,0.0
9,0,700.0,3.92,2,P:2 A:0.0,0.0,1.0,0.0,0.0


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: one per unique row

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [31]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'P_1':])
print handCalc.head()

   admit    gre   gpa  P_1  P_2  P_3  P_4
0      0  380.0  3.61  0.0  0.0  1.0  0.0
1      1  660.0  3.67  0.0  0.0  1.0  0.0
2      1  800.0  4.00  1.0  0.0  0.0  0.0
3      1  640.0  3.19  0.0  0.0  0.0  1.0
4      0  520.0  2.93  0.0  0.0  0.0  1.0


In [60]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'P_1':])
handCalc
for col in ('P_1', 'P_2', 'P_3', 'P_4'):
    print 'Prestige {} odds are {:.0f}%'.format(col, handCalc[handCalc[col] == True]['admit'].mean() * 100)
    
pd.crosstab(df.admit, handCalc.P_1)

Prestige P_1 odds are 54%
Prestige P_2 odds are 36%
Prestige P_3 odds are 23%
Prestige P_4 odds are 18%


P_1,0.0,1.0
admit,,
0,243,28
1,93,33


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

for an applicant from a Prestige1 college the odds of being admitted to grad school are at 54%

#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [43]:
prob1answer = 'Prestige {} odds are {:.0f}%'.format(col, handCalc[handCalc['P_1'] == False]['admit'].mean() * 100)
print prob1answer
againstodds = (100 - 28)
prob1odds = 28
print float(againstodds / prob1odds)

Prestige P_4 odds are 28%
2.0


#### Question 3.3. Calculate the odds ratio.

In [ ]:
odds(success) = p/q = .28/1 = 28%,

that is, the odds of success are 1 to 28.  

    odds(failure) = q/p = .72/.28 = 2.57


#### Question 3.4. Write this finding in a sentenance:

Answer: The odds are 2.5 to 1 that you will NOT be accepted to grad school if you didn't attend a #1 college

#### Question 3.5. Print the cross tab for prestige_4.

In [61]:
pd.crosstab(df.admit, handCalc.P_4)

P_4,0.0,1.0
admit,,
0,216,55
1,114,12


#### Question 3.6. Calculate the OR.

#### Question 3.7. Write this finding in a sentence.

Answer:

## Part 4. Analysis

In [45]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'P_2':])
print data.head()

   admit    gre   gpa  P_2  P_3  P_4
0      0  380.0  3.61  0.0  1.0  0.0
1      1  660.0  3.67  0.0  1.0  0.0
2      1  800.0  4.00  0.0  0.0  0.0
3      1  640.0  3.19  0.0  0.0  1.0
4      0  520.0  2.93  0.0  0.0  1.0


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [46]:
# manually add the intercept
data['intercept'] = 1.0

#### Question 4.1. Set the covariates to a variable called train_cols.

In [85]:
train_cols = ['gre', 'gpa', 'P_2', 'P_3', 'P_4', 'intercept']

def X_y(df):
    X = df[ train_cols ]
    y = df['admit']
    return X, y

train_X, train_y = X_y(data)

#### Question 4.2. Fit the model.

In [89]:
model = linear_model.LogisticRegression()
model.fit(train_X, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Question 4.3. Print the summary results.

In [90]:
print 'intercept    =', model.intercept_
print 'coefficients =', model.coef_

intercept    = [-1.09518906]
coefficients = [[ 0.00188666  0.32452811 -0.62006709 -1.16988125 -1.3847737  -1.09518906]]


#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [92]:
my_odds_ratio = np.exp(model.coef_[0])
my_odds_ratio

array([ 1.00188844,  1.38337769,  0.53790835,  0.3104038 ,  0.25038046,
        0.33447637])

#### Question 4.5. Interpret the OR of Prestige_2.

Answer: the OR can be interpreted as a 1 to 2 chance of success.

#### Question 4.6. Interpret the OR of GPA.

Answer: the OR of the GPA (1.38) can be interpreted as having a 40% increase in odds of admission with every point of gpa increase.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [93]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [107]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### Question 5.1. Recreate the dummy variables.

In [108]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
# keep only what we need for making predictions
combos = combos.join(pd.get_dummies(combos.prestige, prefix='P'))
del combos['prestige']
del combos['P_1.0']
combos.columns = ['gre', 'gpa', 'intercept', 'P_2', 'P_3', 'P_4']
combos

,gre,gpa,intercept,P_2,P_3,P_4
0,220.0,2.260000,1.0,0.0,0.0,0.0
1,220.0,2.260000,1.0,1.0,0.0,0.0
2,220.0,2.260000,1.0,0.0,1.0,0.0
3,220.0,2.260000,1.0,0.0,0.0,1.0
4,220.0,2.453333,1.0,0.0,0.0,0.0
5,220.0,2.453333,1.0,1.0,0.0,0.0
6,220.0,2.453333,1.0,0.0,1.0,0.0
7,220.0,2.453333,1.0,0.0,0.0,1.0
8,220.0,2.646667,1.0,0.0,0.0,0.0
9,220.0,2.646667,1.0,1.0,0.0,0.0


#### Question 5.2. Make predictions on the enumerated dataset.

In [112]:
some_prob_predictions = model.predict_proba(combos)
some_prob

AttributeError: 'numpy.ndarray' object has no attribute 'head'

#### Question 5.3. Interpret findings for the last 4 observations.

Answer:

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.